In [3]:
# Import required libraries for Reddit API data collection
import praw  # For making HTTP requests to Reddit API
import pandas as pd  # For data manipulation and analysis
import re
from datetime import datetime
import emoji
import string
#from pysentimiento.preprocessing import preprocess_tweet
#from pysentimiento import create_analyzer

In [4]:
def get_reddit_instance():
    reddit = praw.Reddit(
        client_id = 'LDPs59MIc82xLXPftHQ3Sw',
        client_secret = "uZSSG2q3ueggUkUReIMzaKe8xnC6iw",
        password = 'Linyihao.041228',
        user_agent = "Comment Extraction (by SaltyfishQAQ)",
        username = 'Few-Strength-2343'
    )
    
    return reddit


def get_monthly_top_posts(reddit, subreddit_name, limit=10):
    subreddit = reddit.subreddit(subreddit_name)
    
    data_df = []
    for submission in subreddit.top(time_filter='month', limit=limit):
        submission_info = {
            "submission_title": submission.title,
            "author": submission.author.name if submission.author else "deleted",
            "id": submission.id,
            "url": submission.url,
            "created_utc": submission.created_utc,
            "body": submission.selftext,  
            "score": submission.score
        }
        data_df.append(submission_info)
        
        if submission.num_comments == 0:
            continue
        else:
            submission.comments.replace_more(limit=None)
            for comment in submission.comments.list():
                # Skip MoreComments objects if any remain
                if isinstance(comment, praw.models.MoreComments):
                    continue
                
                comment_info = {
                    "submission_title": submission.title,
                    "author": comment.author.name if comment.author else "deleted",
                    "id": comment.id,
                    "url": f"https://www.reddit.com{comment.permalink}",
                    "created_utc": comment.created_utc,
                    "body": comment.body,
                    "score": comment.score
                }
                data_df.append(comment_info)
        
    data_df = pd.DataFrame(data_df).reset_index(drop=True)
        
    return data_df

In [5]:
reddit = get_reddit_instance()
df = get_monthly_top_posts(reddit, 'UofT', limit=1)

In [6]:
df.head()

,submission_title,author,id,url,created_utc,body,score
0,Why is UofT using AI?? Could they not have fou...,DecaffeinatedTanya,1mb6vic,https://i.redd.it/4a9gteowljff1.png,1.753677e+09,,930
1,Why is UofT using AI?? Could they not have fou...,internetMujahideen,n5k4y1g,https://www.reddit.com/r/UofT/comments/1mb6vic...,1.753680e+09,They don't make diverse utopian friend groups ...,316
2,Why is UofT using AI?? Could they not have fou...,random_name_245,n5kzzls,https://www.reddit.com/r/UofT/comments/1mb6vic...,1.753697e+09,Lol I wonder what the prompt was like - “Have ...,115
3,Why is UofT using AI?? Could they not have fou...,ihatedougford,n5k06ru,https://www.reddit.com/r/UofT/comments/1mb6vic...,1.753677e+09,"Classic University College, the cheapest and l...",111
4,Why is UofT using AI?? Could they not have fou...,BabaYagaTO,n5n0g1d,https://www.reddit.com/r/UofT/comments/1mb6vic...,1.753722e+09,And now it's gone? [https://www.uc.utoronto.ca...,18


In [7]:
# Data cleaning step

# for the selftexts: 
# lowercase all texts
# Delete spaces in front of and after the texts, keep the spaces in between
# Delete all special characters except for spaces
# Remove all error reading form the api. eg. error languange

# irelevant comments:
# Delete all comments that are not related to UofT: url not start from: https://www.reddit.com/r/UofT/comments/

# Time conveting:
# Convert time format in created_utc to readable format(20XX-XX-XX)
# utc format: scientific format for secs; 1.752931e+09 = 1,757,931,000 sec since Jan 1, 1970 UTC; 
# represent time stamp 2025-07-23 05:03:20

In [8]:
def clean_text(text):
    """
    Remove all special characters from the text except for letters, numbers, and spaces.
    """
    return re.sub(r'[^\w\s]', '', text)


def convert_emoji_to_text(text, emoji_wrapper="emoji"):
    """
    Converts emoji in the text to descriptive text.
    
    This function uses emoji.demojize() to replace any emoji with their
    corresponding text (e.g., "😄" becomes ":smile:"). It then wraps the
    emoji description using the provided emoji_wrapper.
    
    Args:
        text (str): The original text containing emoji.
        emoji_wrapper (str): A string that will be used to wrap the emoji text.
        
    Returns:
        str: The text with emojis converted to wrapped text.
    """
    # Convert emojis to descriptive text (e.g., :smile:)
    demojized = emoji.demojize(text)
    # Define a wrapper string (e.g., " emoji ")
    wrapper = f" {emoji_wrapper} ".replace("  ", " ")
    # Replace the demojized emoji pattern :emoji_name: with the wrapped emoji_name
    # For example, ":smile:" becomes " emoji smile emoji "
    result = re.sub(r':([^:\s]+):', lambda m: wrapper + m.group(1) + wrapper, demojized)
    return result

def is_valid_word(word):
    """
    Returns True if the word is either:
    - Composed solely of punctuation
    - Contains at least one English letter (a-z or A-Z)
    """
    # Keep if word is only punctuation
    if all(ch in string.punctuation for ch in word):
        return True
    # Keep if the word contains at least one ASCII letter
    if re.search(r'[a-zA-Z]', word):
        return True
    return False

def filter_non_english(text):
    """
    Splits the text into words and filters out any word that is not an English word,
    an emoji, or punctuation.
    
    Args:
        text (str): The input text.
    
    Returns:
        str: The cleaned text.
    """
    words = text.split()
    filtered_words = [word for word in words if is_valid_word(word)]
    return ' '.join(filtered_words)

# 4. Convert created_utc to readable date format (YYYY-MM-DD)
def convert_utc_to_readable(utc_timestamp):
    """
    Convert UTC timestamp to a readable date format (YYYY-MM-DD).
    
    Args:
        utc_timestamp (float): UTC timestamp in seconds since epoch.
        
    Returns:
        str: Formatted date string or empty string if input is NaN.
    """
    
    try:
        # Handle NaN values
        if pd.isna(utc_timestamp):
            return ""
        
        # Convert to datetime object
        dt = datetime.fromtimestamp(utc_timestamp)
        
        # Format as YYYY-MM-DD HH:MM:SS
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    
    except (ValueError, TypeError, OSError):
        return ""
    

In [9]:
# 1. Keep only posts with URLs starting with UofT comments
df = df[df['url'].str.startswith('https://www.reddit.com/r/UofT/comments/')].reset_index(drop=True).copy()

# 2. Clean selftext: lowercase, strip, remove special chars except spaces
df['body'] = df['body'].str.lower()
df['body'] = df['body'].str.strip()

# 3. Remove special characters except spaces
df['body'] = df['body'].apply(clean_text)

# 4. Converts emoji in the text to descriptive text.
df['body'] = df['body'].apply(convert_emoji_to_text)

# 5. Filter out non-English words
df['body'] = df['body'].apply(filter_non_english)

# 6. Convert created_utc to readable date format
df['created_utc'] = df['created_utc'].apply(convert_utc_to_readable)

# 7. Preprocess the selftext column using pysentimiento
# df['body'] = df['body'].apply(preprocess_tweet)

df.head()

,submission_title,author,id,url,created_utc,body,score
0,Why is UofT using AI?? Could they not have fou...,internetMujahideen,n5k4y1g,https://www.reddit.com/r/UofT/comments/1mb6vic...,2025-07-28 01:14:54,they dont make diverse utopian friend groups s...,316
1,Why is UofT using AI?? Could they not have fou...,random_name_245,n5kzzls,https://www.reddit.com/r/UofT/comments/1mb6vic...,2025-07-28 06:07:07,lol i wonder what the prompt was like have stu...,115
2,Why is UofT using AI?? Could they not have fou...,ihatedougford,n5k06ru,https://www.reddit.com/r/UofT/comments/1mb6vic...,2025-07-28 00:35:56,classic university college the cheapest and la...,111
3,Why is UofT using AI?? Could they not have fou...,BabaYagaTO,n5n0g1d,https://www.reddit.com/r/UofT/comments/1mb6vic...,2025-07-28 13:00:11,and now its gone httpswwwucutorontocaprogramss...,18
4,Why is UofT using AI?? Could they not have fou...,CaptainKoreana,n5kqi34,https://www.reddit.com/r/UofT/comments/1mb6vic...,2025-07-28 04:36:00,yikes uc,13


In [ ]:


sentiment_analyzer = create_analyzer(task="sentiment", lang="en")

def sentiment_analysis(df, text_column='selftext'):
    """
    Iterates through each selftext in the DataFrame, applies emotion_analyzer,
    and adds two columns: sentiment_prediction and sentiment_prob.
    """
    senti_pred = []
    senti_prob = []
    
    for index, row in df.iterrows():
        text = row[text_column]
        
        if pd.isna(text) or text == "":
            senti_pred.append("NEU")
            senti_prob.append({"NEU": 0.34, "NEG": 0.33, "POS": 0.33})
        else:
            result = sentiment_analyzer.predict(text)
            senti_pred.append(result.output)
            senti_prob.append(result.probas)
    
    df['sentiment_prediction'] = senti_pred
    df['sentiment_prob'] = senti_prob
    
    return df

# Usage:
results_df = sentiment_analysis(df, text_column='body')
results_df.head()
# Export the DataFrame to an Excel file on your Desktop
results_df.to_excel("/Users/terrietaw/Desktop/uoft_sentiment_results.xlsx", index=False)